**Automated Sales Outreach**

*What this will be:*

A workflow of Agent call

An agent that can use tool

An agent that can call on other agents tools and, Vs Handoffs


In [3]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from openai.types.responses import ResponseTextDeltaEvent
from openai import AsyncOpenAI
from typing import Dict
import brevo_python
import os
from brevo_python.rest import ApiException
from brevo_python import SendSmtpEmail, ApiClient, Configuration, TransactionalEmailsApi
import logging
import asyncio



In [4]:
configuration = brevo_python.Configuration()
configuration.api_key['api-key'] = os.getenv("BREVO_API_KEY")

api_instance = brevo_python.AccountApi(brevo_python.ApiClient(configuration))

In [4]:
def send_test_email(to_email, subject, html_content, sender_email="jhayar.zuniga004@gmail.com", sender_name="Jay Test"):
    config = Configuration()
    config.api_key['api-key'] = os.getenv("BREVO_API_KEY")

    api_client = ApiClient(configuration=config)
    api_instance = TransactionalEmailsApi(api_client)

    email = SendSmtpEmail(
        to=[{"email": to_email}],
        sender={"email": sender_email, "name": sender_name},
        subject=subject,
        html_content=html_content,
    )

    try:
        response = api_instance.send_transac_email(email)
        print("✅ Email sent! Message ID:", response.message_id)
    except ApiException as e:
        print("❌ Exception when sending email:", e)

In [32]:
send_test_email(
    to_email="jhayar.zuniga@gmail.com",
    subject="Test Email Subject",
    html_content="<p>This is a test email sent using Brevo API.</p>",
)

✅ Email sent! Message ID: <202507220747.76666306187@smtp-relay.mailin.fr>


In [6]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [7]:
deepseek_api_key = os.getenv("DEEPSEEK_API_KEY")
DEEPSEEK_BASE_URL = "https://api.deepseek.com"

deepseek_client=AsyncOpenAI(
    base_url=DEEPSEEK_BASE_URL,
    api_key=deepseek_api_key
)
deepseek_model=OpenAIChatCompletionsModel(
    model="deepseek-chat",
    openai_client=deepseek_client
)


sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=deepseek_model
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=deepseek_model
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=deepseek_model
)

In [7]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

**Subject:** Simplify SOC2 Compliance with AI-Powered Automation  

Hi [First Name],  

I hope this email finds you well. I wanted to reach out because I know how time-consuming and complex SOC2 compliance can be—especially when preparing for audits.  

At ComplAI, we’ve built an AI-powered SaaS platform that automates SOC2 compliance, saving teams like yours countless hours and reducing the stress of audit prep. Our solution helps you:  
- **Streamline evidence collection** with AI-driven workflows  
- **Identify and close compliance gaps** proactively  
- **Generate audit-ready reports** in minutes, not weeks  

I’d love to show you how ComplAI can simplify your compliance process. Would you be open to a quick 15-minute demo to see it in action?  

Let me know a time that works for you, or feel free to reply with any questions.  

Best regards,  
[Your Full Name]  
[Your Job Title]  
ComplAI  
[Your Email] | [Your Phone Number]  
[Website URL]  

P.S. Many of our customers cut their 

In [9]:


message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

**Subject:** Simplify SOC2 Compliance with AI-Powered Automation  

Hi [First Name],  

I hope this email finds you well. I’m reaching out because I know how time-consuming and complex SOC2 compliance can be—especially when preparing for audits. Manual processes, scattered documentation, and last-minute scrambles can turn compliance into a major headache.  

At ComplAI, we’ve built an AI-powered SaaS platform that automates SOC2 compliance, saving teams like yours hundreds of hours while ensuring audit readiness. Our solution helps you:  

- **Automate evidence collection** – No more chasing down teams for documentation.  
- **Close compliance gaps proactively** – AI identifies risks before they become audit issues.  
- **Streamline audit prep** – Generate audit-ready reports in minutes, not weeks.  

Companies using ComplAI have cut their compliance workload by 60% and passed audits with confidence. I’d love to show you how we can simplify your SOC2 journey—would you be open to a quic

In [14]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with just the selected email only.",
    model=deepseek_model
)

In [15]:
message = "Write a cold sales email without explanations"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")

Best sales email:
**Subject:** Simplify SOC2 Compliance with AI-Powered Automation  

Hi [First Name],  

Ensuring SOC2 compliance can be a complex, time-consuming process—especially when preparing for audits. At ComplAI, we’ve built an AI-driven SaaS solution that automates compliance workflows, reduces manual effort, and helps you stay audit-ready year-round.  

Our platform enables you to:  
- **Automate evidence collection** and documentation  
- **Identify and close compliance gaps** proactively  
- **Streamline audit preparation** with real-time monitoring  

I’d love to show you how ComplAI can save your team hours of work while reducing compliance risks. Are you available for a quick 15-minute demo next week?  

Best regards,  
[Your Name]  
[Your Position]  
ComplAI  
[Your Email] | [Your Phone]  
[Website URL]  

P.S. Many companies cut their compliance prep time by 50% or more—let’s discuss how we can do the same for you.


In [ ]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model=deepseek_model,
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model=deepseek_model,
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model=deepseek_model,
)

In [24]:
# @function_tool
# def send_email(body: str):
#     """ Send out an email with the given body to all sales prospects """
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
#     from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
#     to_email = To("ed.donner@gmail.com")  # Change to your recipient
#     content = Content("text/plain", body)
#     mail = Mail(from_email, to_email, "Sales email", content).get()
#     sg.client.mail.send.post(request_body=mail)
#     return {"status": "success"}

@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    config = Configuration()
    config.api_key['api-key'] = os.getenv("BREVO_API_KEY")

    api_client = ApiClient(configuration=config)
    api_instance = TransactionalEmailsApi(api_client)

    email = SendSmtpEmail(
        to=[{"email": "jhayar.zuniga@gmail.com"}],
        sender={"email": "jhayar.zuniga004@gmail.com", "name": "Jay Z"},
        subject="Sales Agent SOC1",
        html_content= body,
    )

    try:
        response = api_instance.send_transac_email(email)
        print("✅ Email sent! Message ID:", response.message_id)
    except ApiException as e:
        print("❌ Exception when sending email:", e)

In [25]:
send_email

FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026AF7CC4E00>, strict_json_schema=True, is_enabled=True)

TOOLS

In [26]:
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000026AF9C0D580>, strict_json_schema=True, is_enabled=True)

In [17]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000011F2AA82700>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000011F2AA81440>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'

In [ ]:
instructions ="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales_agent tools once before choosing the best one. \
You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

# with trace("Sales manager"):
#     result = await Runner.run(sales_manager, message)

Error getting response: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}. (request_id: req_c9082c5192b5a12c3ccd13c7aee0347c)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

HANDOFFS

In [9]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=deepseek_model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model=deepseek_model)
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [10]:
# @function_tool
# def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
#     """ Send out an email with the given subject and HTML body to all sales prospects """
#     sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
#     from_email = Email("ed@edwarddonner.com")  # Change to your verified sender
#     to_email = To("ed.donner@gmail.com")  # Change to your recipient
#     content = Content("text/html", html_body)
#     mail = Mail(from_email, to_email, subject, content).get()
#     sg.client.mail.send.post(request_body=mail)
#     return {"status": "success"}

@function_tool
def send_email(subject: str, body: str) -> Dict[str, str]:
    """ Send out an email with the given body to all sales prospects """
    config = Configuration()
    config.api_key['api-key'] = os.getenv("BREVO_API_KEY")

    api_client = ApiClient(configuration=config)
    api_instance = TransactionalEmailsApi(api_client)

    email = SendSmtpEmail(
        to=[{"email": "jhayar.zuniga@gmail.com"}],
        sender={"email": "jhayar.zuniga004@gmail.com", "name": "Jay Z"},
        subject="Sales Agent SOC1",
        text_content= f"Subject: {subject}\n\n{body}",
    )

    try:
        response = api_instance.send_transac_email(email)
        print("✅ Email sent! Message ID:", response.message_id)
    except ApiException as e:
        print("❌ Exception when sending email:", e)

In [12]:
tools = [subject_tool, html_tool, send_email]

In [13]:
tools

[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000011F2AA81A80>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000011F2AA81EE0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='send_email', description='Send out an email with the given body to all sales prospects', 

In [15]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [18]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]
print(tools)
print(handoffs)

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000011F2AA82700>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000011F2AA81440>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}

In [ ]:
sales_manager_instructions = "You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails. \
You never generate sales emails yourself; you always use the tools. \
You try all 3 sales agent tools at least once before choosing the best one. \
You can use the tools multiple times if you're not satisfied with the results from the first try. \
You select the single best email using your own judgement of which email will be most effective. \
After picking the email, you handoff to the Email Manager agent to format and send the email."


sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

result = await Runner.run(sales_manager, message)

# with trace("Automated SDR"):
#     result = await Runner.run(sales_manager, message)